In [1]:
import os
#if the directory is not the root of the project, change it
os.chdir('/home/mila/e/emiliano.penaloza/RLPHF')
import torch.nn as nn 
from datasets import Dataset
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
# %%
from transformers import BertTokenizer, BertModel
import torch
from torch.nn import MultiheadAttention
from transformers import LlamaModel, LlamaConfig,LlamaForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

from notebooks.hypernet import HyperNetController, HyperNet, HyperNetLinear
os.environ['TRANSFORMERS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_DATASETS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['TORCH_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
cache_dir = '/home/mila/e/emiliano.penaloza/scratch/models'


# %%
import copy 
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer



/home/mila/e/emiliano.penaloza/rlhf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
r = 1
alpha = 0.1
dropout = 0.1
n_transformer_layers = 1
n_transformer_heads = 2
d_emb = 64
d_model = 16
a_b = 2 
kvq =3
#this is to produce a layer at a time
output_dim = (r *  d_model ) * a_b  * kvq   
a = 16 


# Load the datasets
sst2 = load_dataset("nyu-mll/glue", "sst2")
mrpc = load_dataset("nyu-mll/glue", "mrpc")

# Function to process MRPC dataset
def process_mrpc(example):
    return {
        "text": example["sentence1"] + " " + example["sentence2"],
        "label": example["label"],
        "dataset": "mrpc"
    }

# Process MRPC dataset (only the 'train' split)
mrpc_processed = mrpc['train'].map(process_mrpc)

# Function to process SST2 dataset
def process_sst2(example):
    return {
        "text": example["sentence"],
        "label": example["label"],
        "dataset": "sst2"
    }

# Process SST2 dataset (only the 'train' split)
sst2_processed = sst2['train'].map(process_sst2)

# Convert both datasets to pandas DataFrames
df_mrpc = mrpc_processed.to_pandas()
df_sst2 = sst2_processed.to_pandas()

# Calculate the number of samples to keep for SST2
n_mrpc = len(df_mrpc)
n_sst2_to_keep = int(n_mrpc * 1.5)  # 60% SST2, 40% MRPC

# Downsample SST2
df_sst2_downsampled = df_sst2.sample(n=n_sst2_to_keep, random_state=42)

# Combine the DataFrames
df_combined = pd.concat([df_mrpc, df_sst2_downsampled], ignore_index=True)

# Ensure all columns have the same dtype
df_combined["text"] = df_combined["text"].astype(str)
df_combined["label"] = df_combined["label"].astype(int)
df_combined["dataset"] = df_combined["dataset"].astype(str)

# Create a stratified train-test split
train_df, test_df = train_test_split(df_combined, test_size=0.2, stratify=df_combined['dataset'], random_state=42)

# Shuffle the datasets
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert back to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Print info about the final datasets
print("Train dataset:")
print(train_dataset)
print("\nTest dataset:")
print(test_dataset)

# Verify the distribution of tasks in each split
print("\nTask distribution in train set:")
print(train_df['dataset'].value_counts(normalize=True))
print("\nTask distribution in test set:")
print(test_df['dataset'].value_counts(normalize=True))


#subset the train_dataset to only have text label and dataset 
train_dataset = train_dataset.remove_columns(['idx'])
test_dataset = test_dataset.remove_columns(['idx'])

train_dataset = train_dataset.remove_columns(['sentence1','sentence2','sentence'])
test_dataset = test_dataset.remove_columns(['sentence1','sentence2','sentence'])

# %%
#laod train_df into a dataloader 


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# %%

# Define the model name
model_name = "bert-base-uncased"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model loaded: {model_name}")
print(f"Model is on device: {device}")

# %%
class_encodings = nn.Parameter(torch.empty(2,64))

nn.init.xavier_normal_(class_encodings)
print('initialized')

# %%

r = 8
alpha = 0.1
dropout = 0.1
n_transformer_layers = 1
n_transformer_heads = 2
d_emb = 64
d_model = 16
a_b = 2 
kvq =3
#this is to produce a layer at a time
output_dim = (r *  d_model ) * a_b  * kvq   
a = 16 

#load bert as the model 
# model = BertModel.from_pretrained('bert-base-uncased',cache_dir  = cache_dir).to(0)
A = model.config.hidden_size

import torch
import wandb
from torch.utils.data import DataLoader
from tqdm import tqdm

# Initialize wandb
wandb.init(project="your_project_name", name="your_run_name")

# Define hyperparameters
hyperparameters = {
    "learning_rate": 1e-4,
    "epochs": 1000,
    "batch_size": 32,
    "r": r,
    "alpha": alpha,
    "dropout": dropout,
    "output_dim": output_dim,
    "d_emb": d_emb,
    "d_model": d_model,
    "n_transformer_layers": n_transformer_layers,
    "n_transformer_heads": n_transformer_heads,
    "a": a,
    "A": A
}

wandb.config.update(hyperparameters)



Train dataset:
Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'text', 'dataset', 'sentence'],
    num_rows: 7336
})

Test dataset:
Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'text', 'dataset', 'sentence'],
    num_rows: 1834
})

Task distribution in train set:
dataset
sst2    0.600055
mrpc    0.399945
Name: proportion, dtype: float64

Task distribution in test set:
dataset
sst2    0.599782
mrpc    0.400218
Name: proportion, dtype: float64


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded: bert-base-uncased
Model is on device: cuda
initialized


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: emilianopp (emilianouw). Use `wandb login --relogin` to force relogin


In [5]:
def evaluate(model, hypernet, controller, eval_loader, task_mapper, device):
    model.eval()
    hypernet.eval()
    total_loss = 0

    # Initialize dictionaries to keep track of predictions per task
    task_correct_predictions = {}
    task_total_predictions = {}

    with torch.no_grad():
        for b in eval_loader:
            encodings = tokenizer(b['text'], truncation=True, padding=True, return_tensors='pt')
            labels = b['label'].to(device)
            input_ids = encodings['input_ids'].to(device)
            attention_mask = encodings['attention_mask'].to(device)
            tasks = [task_mapper[task] for task in b['dataset']]

            task_embedding = torch.stack([class_encodings[task] for task in tasks]).unsqueeze(1).repeat(1,12,1).to(device)
            identity_matrix = torch.eye(d_model).unsqueeze(0).repeat(input_ids.shape[0], 1, 1).to(device)[:,:12,:]
            hypernet_layers = hypernet(task_embedding, identity_matrix).view(input_ids.shape[0], n_layers*kvq, r, -1)
            controller.updateLayers(hypernet_layers)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            predictions = torch.argmax(outputs.logits, dim=-1)

            # Update task-specific predictions
            for task, pred, label in zip(tasks, predictions, labels):
                if task not in task_correct_predictions:
                    task_correct_predictions[task] = 0
                    task_total_predictions[task] = 0
                task_correct_predictions[task] += (pred == label).item()
                task_total_predictions[task] += 1

    avg_loss = total_loss / len(eval_loader)

    # Calculate accuracy per task
    task_accuracy = {task: task_correct_predictions[task] / task_total_predictions[task] for task in task_correct_predictions}

    return avg_loss, task_accuracy




In [6]:
# Set up model, hypernet, and controller
hypernet = HyperNet(r, alpha, dropout, output_dim, d_emb, d_model, n_transformer_layers, n_transformer_heads, a=a).to(0)
controller = HyperNetController(hypernet, target_modules=['query', 'key', 'value'], r=r, A=A, a=a)
controller.augmentLLM(model)
controller.freezeParams(model, False)

# Set up optimizers
# model_trainable_params = [param for param in model.parameters() if param.requires_grad]
#hypernet and calss encodings
hypernetParameters = [param for param in hypernet.parameters() if param.requires_grad] + [class_encodings]
# modelOptimizer = torch.optim.Adam(model_trainable_params, lr=hyperparameters['learning_rate'])
hypernetOptimizer = torch.optim.AdamW(hypernetParameters, lr=hyperparameters['learning_rate'])

# Evaluation function


In [7]:
labels = test_df[test_df['dataset'] == 'sst2'].label
prop_1 = sum(labels)/len(labels)
prop_0 = 1 - prop_1
prop_1, prop_0

(0.5581818181818182, 0.4418181818181818)

In [8]:
labels = test_df[test_df['dataset'] == 'mrpc'].label
prop_1 = sum(labels)/len(labels)
prop_0 = 1 - prop_1
prop_1, prop_0

(0.6771117166212534, 0.32288828337874664)

In [11]:
# Training loop
task_mapper = {'mrpc': 0, 'sst2': 1}
device = 'cuda'
n_layers = 12
for epoch in range(hyperparameters['epochs']):
    model.train()
    hypernet.train()
    total_train_loss = 0

    for k,b in tqdm(enumerate(train_loader), desc=f"Epoch {epoch+1}/{hyperparameters['epochs']}"):
        # modelOptimizer.zero_grad()
        hypernetOptimizer.zero_grad()

        encodings = tokenizer(b['text'], truncation=True, padding=True, return_tensors='pt')
        labels = b['label'].to(device)
        input_ids = encodings['input_ids'].to(device)
        attention_mask = encodings['attention_mask'].to(device)
        tasks = [task_mapper[task] for task in b['dataset']]


        task_embedding = torch.stack([class_encodings[task] for task in tasks]).unsqueeze(1).repeat(1,12,1).to(device)


        identity_matrix = torch.eye(d_model,requires_grad = False).unsqueeze(0).repeat(input_ids.shape[0], 1, 1).to(device)[:,:12,:]
        hypernet_layers = hypernet(task_embedding, identity_matrix).view(input_ids.shape[0], n_layers*kvq, r, -1)

        controller.updateLayers(hypernet_layers)

        loss = model(input_ids, attention_mask=attention_mask, labels=labels).loss
        total_train_loss += loss.item()

        loss.backward()

      
        # modelOptimizer.step()
        hypernetOptimizer.step()
        # if k == 0:
        
    eval_loss, eval_accuracy = evaluate(model, hypernet, controller, test_loader, task_mapper, device)

    avg_train_loss = total_train_loss / len(train_loader)
    
    # Evaluation

    # Log metrics to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "eval_loss": eval_loss
    })

    print(f"Epoch {epoch+1}/{hyperparameters['epochs']}")
    print(f"Train Loss: {avg_train_loss:.4f}")
    print(f"Eval Loss: {eval_loss:.4f}")
    print(f"Eval Accuracy: {eval_accuracy}")





Epoch 1/1000: 0it [00:00, ?it/s]

task_embedding.shape=torch.Size([32, 12, 64])


Exception: 

In [ ]:
#get number of  trainable parameters for hypernet 

total_params = sum(p.numel() for p in hypernet.parameters())
print("Total number of parameters for the hypernet:", total_params)

Total number of parameters for the hypernet: 96992


In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',cache_dir  = cache_dir).to(0)
#add peft lora model


In [ ]:
for n,m in model.named_parameters():
    print(n)
    print(m.requires_grad)
    print('\n')


bert.embeddings.word_embeddings.weight
False


bert.embeddings.position_embeddings.weight
False


bert.embeddings.token_type_embeddings.weight
False


bert.embeddings.LayerNorm.weight
False


bert.embeddings.LayerNorm.bias
False


bert.encoder.layer.0.attention.self.query.weight
False


bert.encoder.layer.0.attention.self.query.bias
False


bert.encoder.layer.0.attention.self.query.hyperAdapterA
True


bert.encoder.layer.0.attention.self.query.hyperAdapterB
True


bert.encoder.layer.0.attention.self.key.weight
False


bert.encoder.layer.0.attention.self.key.bias
False


bert.encoder.layer.0.attention.self.key.hyperAdapterA
True


bert.encoder.layer.0.attention.self.key.hyperAdapterB
True


bert.encoder.layer.0.attention.self.value.weight
False


bert.encoder.layer.0.attention.self.value.bias
False


bert.encoder.layer.0.attention.self.value.hyperAdapterA
True


bert.encoder.layer.0.attention.self.value.hyperAdapterB
True


bert.encoder.layer.0.attention.output.dense.weight
False


bert